In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%cd ..

In [ ]:
from src import locations
import duckdb
from jinja2 import Template
import os

In [ ]:
create_view_sql = Template(
    """
    CREATE VIEW all_taxi AS 
    SELECT * 
    FROM read_parquet('{{ raw_data_dir }}/nyc_taxi/**/*.parquet', hive_partitioning=true)
    """
).render(raw_data_dir=locations.DATA_SUBDIR["raw"])
print(create_view_sql)

In [ ]:
with duckdb.connect() as duck:
    duck.execute(create_view_sql)
    duck.sql("SELECT * FROM all_taxi LIMIT 10").show()

In [ ]:
with duckdb.connect() as duck:
    duck.execute(create_view_sql)
    weekly_summary = duck.sql(
        """
        SELECT 
          week(tpep_pickup_datetime) AS week,
          year,
          count(*) as trips,
          AVG(total_amount) AS avg_cost
        FROM all_taxi
        GROUP BY week(tpep_pickup_datetime), year
        """
    ).pl()
weekly_summary
weekly_summary.write_csv(
    os.path.join(locations.DATA_SUBDIR["processed"], "weekly_summary.csv")
)

In [ ]:
duck = duckdb.connect(database=":memory:", read_only=False)

In [ ]:
duck.sql(
    "CREATE VIEW all_taxi AS SELECT * FROM read_parquet('data/raw/nyc_taxi/**/*.parquet', hive_partitioning = true)"
)

In [ ]:
duck.sql(
    "SELECT month, COUNT(*) AS N FROM all_taxi WHERE year = 2020 GROUP BY month ORDER BY month"
).show()

In [ ]:
# duckdb query to find the number of trips per month in 2020
duck.sql(
    "SELECT month, COUNT(*) AS N FROM all_taxi WHERE year = 2020 GROUP BY month ORDER BY month"
).show()

In [ ]:
# duckdb query to find the number of trips per month in 2020, return result as polars data frame
duck.sql("SELECT * FROM all_taxi LIMIT 10").pl()

In [ ]:
duck.close()

In [ ]:
with duckdb.connect() as duck: